In [12]:
! pip install -Uq fastai fastbook setuptools ipywidgets RandomWords gradio

In [13]:
%matplotlib inline
#This helps widgets somehow

In [14]:
#hide
import fastbook
fastbook.setup_book()
import pathlib   
from pathlib import Path

In [15]:
#hide
from fastbook import *
from fastai.vision.widgets import *

In [16]:
# If you want a spicific model version, then add /'version number' or 'latest' to the path
modelFolder = Path.cwd().parent.parent / 'Trained-models'
print(modelFolder)

/Users/victor/Documents/GitHub/VS-Code/Side-Projects/Trained-models


In [17]:
import os
def find_files_with_extension(file_extension):
    matching_files = [file for file in os.listdir(modelFolder) if file.endswith(file_extension)]
    return matching_files

# Example usage
file_type = '.pkl'  # Change this to the file type you're looking for
matching_files = find_files_with_extension(file_type)

if matching_files:
    print(f"This directory contains the following '{file_type}' file(s):")
    for file in matching_files:
        print(f"- {file}")
else:
    print(f"This directory does not contain any '{file_type}' files")

This directory contains the following '.pkl' file(s):
- countryGuesser.pkl
- animalClassifier.pkl
- leagueChampionClassifier.pkl
- politicianClassifier.pkl
- test.pkl


In [18]:
# exclude '.pkl'
modelName = 'countryGuesser'

import os
if os.name == 'nt':
    print(os.name)
    # Changes PosixPath to WindowsPath
    temp = pathlib.PosixPath   
    pathlib.PosixPath = pathlib.WindowsPath
else:
    print(os.name)
    temp = pathlib.PosixPath   
    pathlib.WindowsPath = pathlib.PosixPath

modelPath = modelFolder / f'{modelName}.pkl'
print(modelPath)

learn_inf = load_learner(modelPath)
print("Model loaded successfully!")

posix
/Users/victor/Documents/GitHub/VS-Code/Side-Projects/Trained-models/countryGuesser.pkl
Model loaded successfully!


In [19]:
#hide_output
btn_upload = widgets.FileUpload()
out_pl = widgets.Output()
lbl_pred = widgets.Label()
btn_run = widgets.Button(description='Classify')

In [20]:
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(448,448))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.02f}'

btn_run.on_click(on_click_classify)

In [21]:
# Alternative test here
VBox([btn_upload, btn_run, out_pl, lbl_pred])

In [22]:
import gradio as gr
from fastai.vision.all import *


learn = learn_inf
labels = learn.dls.vocab
def predict(img):
    img = PILImage.create(img)
    pred,pred_idx,probs = learn.predict(img)
    return {labels[i]: float(probs[i]) for i in range(len(labels))}

interface = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil"),
    outputs=gr.Label(num_top_classes=10),
    title=modelName,
    description="")

interface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://d4261daf99bf7b3689.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
